In [132]:
import random
from datetime import datetime, date, timedelta
import pandas as pd
import itertools
import json
import numpy
import pm4py
import os
import functools

In [133]:
def getRandomTime(min_x, max_x, duration, start_time):
    if type(start_time) == type('a'):
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S:%f")
    random_duration = start_time + timedelta(minutes=duration)
    return (random_duration.strftime('%Y-%m-%d %H:%M:%S:%f'), duration)

In [134]:
def calculateDuration(df, start_time):
    for index, row in df.iterrows():
        if index != 0:
            previous = df.loc[index -1, "End_time"]
            a = getRandomTime(row["Min"],row["Max"], row["Duration"], previous)
            df.at[index, "start_time"] = previous
            df.at[index, "End_time"] = datetime.strptime(a[0], "%Y-%m-%d %H:%M:%S:%f")
            df.at[index, "Duration"] = a[1]
        else:
            a = getRandomTime(row["Min"],row["Max"], row["Duration"], start_time)
            df.at[index, "start_time"] = datetime.strptime(start_time.strftime('%Y-%m-%d %H:%M:%S:%f'), "%Y-%m-%d %H:%M:%S:%f")
            df.at[index, "End_time"] = datetime.strptime(a[0], "%Y-%m-%d %H:%M:%S:%f")
            df.at[index, "Duration"] = a[1]
    return(df)

In [135]:
# Each Activity have 3 properties: Activity, minimum time, maximum time

def haveDinner():
    return ['Dinner', '00:40:00:00','01:10:00:00']

def haveDinner2():
    return ['Dinner', '00:20:00:00','01:30:00:00']

def haveDinnerBreakStats():
    return ['Dinner', '00:30:00:00','01:20:00:00']

In [136]:
def watchTV():
    return ['TV', '00:30:00:00', '01:00:00:00']
 
def watchTV2():
    return ['TV', '00:20:00:00', '01:20:00:00']

def watchTVBreakStats():
    return ['TV', '00:40:00:00', '02:00:00:00']

In [137]:
def goToGym():
    return ['TV', '00:30:00:00', '01:00:00:00']

In [138]:
def brushTeeth():
    return ['Brush teeth', '00:10:00:00', '00:20:00:00']
 
def BrushTeeth2():
    return ['Brush teeth', '00:10:00:00', '00:40:00:00']

def BrushTeethBreakStats():
    return ['Brush teeth', '00:05:00:00', '00:40:00:00']

In [139]:
def read():#21:15
    return ['Read', '00:00:30:00','01:00:00:00']

def read2():
    return ['Read', '00:20:00:00','01:20:00:00']

def readBreakStats():
    return ['Read', '00:01:00:00','00:40:00:00']

In [140]:
def goToSleep(): #Around 22:00
    return ['Sleep', '00:10:00:00','00:30:00:00']

def goToSleep2():
    return ['Sleep', '00:10:00:00','00:30:00:00']

def goToSleepBreakStats():
    return ['Sleep', '00:01:00:00','00:15:00:00']

In [141]:
def waitingTime3A():
    return ['waiting', '00:05:00:00', '00:15:00:00']

def waitingTime3B():
    return ['waiting', '00:01:00:00', '00:15:00:00']

def doNothing():
    return ['waiting', '00:10:00:00', '00:30:00:00']

In [142]:
def build3A():
    activities = []
    group = [watchTV(), read(), doNothing(), doNothing(), doNothing()]

#     Start with dinner
    activity = haveDinner()
    activity_duration = round(random.uniform(40,70),2)
    activities.append([activity, activity_duration])
    activities.append([waitingTime3A(), round(random.uniform(5, 15),2)])
    
#     2x Random activities
    iterations = random.randint(1, 2)
    for i in range(1,iterations+1):
        activity = random.choice(group)
        if activity == doNothing():
            activity_duration = round(random.uniform(10,30),2)
            group.remove(activity)
        else:
            activity_duration = round(random.uniform(30,60),2)
        activities.append([activity, activity_duration])
    activities.append([waitingTime3A(), round(random.uniform(5, 15),2)])
    
#     Brush teeth
    activity = brushTeeth()
    activity_duration = round(random.uniform(10,20),2)
    activities.append([activity, activity_duration])
    
#     2x Random activities
    iterations = random.randint(1, 2)
    for i in range(1,iterations+1):
        activity = random.choice(group)
        if activity == doNothing():
            activity_duration = round(random.uniform(10,30),2)
            group.remove(activity)
        else:
            activity_duration = round(random.uniform(30,60),2)
        activities.append([activity, activity_duration])
    activities.append([waitingTime3A(), round(random.uniform(5, 15),2)])
    
#     Always end by going to sleep
    activity = goToSleep()
    activity_duration = round(random.uniform(10,30),2)
    activities.append([activity, activity_duration])

    return activities

In [143]:
def build3B():    
    eveningDuration = 240 #From 19:xx for four hours
    activities = []
    group = [#haveDinner(),
             watchTV(),
             read(),
             brushTeeth(), 
             goToSleep(),
             goToGym()]
    
    x = [random.choice(group)]
    activity = random.choice(x)

# #     Start with one activity, most likely dinner
#     if activity == haveDinner():
#         activity_duration = round(random.uniform(20,90),2)
#         activities.append([activity, activity_duration])
#         Filter haveDinner from the list used to select next activity
#         group = list(filter((haveDinner()).__ne__, group))
    if activity == brushTeeth():
        activity_duration = round(random.uniform(10,20),2)
        activities.append([activity, activity_duration])
#         group = list(filter((brushTeeth()).__ne__, group))
        group.append(brushTeeth())
    elif activity == goToSleep():
        activity_duration = round(random.uniform(10,30),2)
        activities.append([activity, activity_duration])
#         return activities
    else:
        activity_duration = round(random.uniform(30,60),2)
        activities.append([activity, activity_duration])
    eveningDuration -= activity_duration
 
    group = [haveDinner(),
             watchTV(),
             read(),
             brushTeeth(), 
             goToSleep()]
    
    while eveningDuration > 0:
        activities.append([waitingTime3A(), round(random.uniform(5, 15),2)])

#         iterations = random.randint(1, 2)
#         for i in range(1,iterations+1):
        activity = random.choice(group)
        if activity == haveDinner():
            activity_duration = round(random.uniform(40,70),2)
            activities.append([activity, activity_duration])
#             group = list(filter((haveDinner()).__ne__, group))
        elif activity == brushTeeth():
            activity_duration = round(random.uniform(10,20),2)
            activities.append([activity, activity_duration])
#             group = list(filter((brushTeeth()).__ne__, group))
#             group.append(brushTeeth())
        elif activity == goToSleep():
            break
        else:
            activity_duration = round(random.uniform(30,60),2)
            activities.append([activity, activity_duration])
#         activities.append([waitingTime3A(), round(random.uniform(5, 15),2)])
        eveningDuration -= activity_duration
    
    activity = goToSleep()
    activity_duration = round(random.uniform(10,30),2)
    activities.append([activity, activity_duration])
    return activities

In [144]:
def mergeData(day):
    droprow = []
    result = day
    label_index = 0
    for i in range(1, day.shape[0]):
        if day.iloc[i]['concept:name'] == result.iloc[label_index]['concept:name']:
            result.iloc[label_index, 3] = day.iloc[i, 3]
            result.iloc[label_index, 4] += day.iloc[i, 4]
            droprow.append(i)
        else:
            label_index = i
    result = result.drop(index=droprow)
    result = result.reset_index()
    return result

In [145]:
dayActivities = build3B()
dayActivities.append([haveDinner(), round(random.uniform(1, 15),2)])
dd = pd.DataFrame(dayActivities)
dd = pd.concat([dd[0].apply(lambda x: pd.Series(x)), dd[1]], axis=1)

In [146]:
def createEventLog3A(dayRange, fileName):
    path = './'+fileName.split('.')[0] 
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
        
    daySimulation = pd.DataFrame()

#     Setting the start date (the first date in log) 
    start_time = datetime(2022, 1, 1, 19, 0, 0, 0)
    
    for i in range(dayRange):
        # Create daily routine
        dayActivities = build3A()
#         dayActivities.append([wakeUp(), round(random.uniform(1, 15),2)])
        dd = pd.DataFrame(dayActivities)
        dd = pd.concat([dd[0].apply(lambda x: pd.Series(x)), dd[1]], axis=1)
        dd.columns = ['Activity', 'Min', 'Max', 'Duration']
        dd = dd.reindex(columns = ["Activity","Min", "Max", "start_time", "End_time", "Duration"]) 
        dd = dd.fillna("")

        # Randomly get time and calculate routine
#         start_time = datetime(2022, 1, i+1, 22, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S:%f')
#         Adding one day to the starting date
        start_time = start_time + timedelta(days=1)

        ddfinal = calculateDuration(dd, start_time)
        ddfinal = ddfinal.drop(columns =['Min', 'Max'])
        ddfinal = ddfinal.rename(columns={'Activity':'concept:name'})
        ddfinal = ddfinal.rename(columns={'End_time':'time:timestamp'})
        #ddfinal = ddfinal.rename(columns={'index':'case_id'})
        ddfinal['case_id'] = i+1
        # print(ddfinal)
        ddfinal.insert(loc=0, column='case:concept:name', value=i+1)
        #resfinal = resfinal.reindex(columns = ['case:concept:name', 'concept:name', 'Start_time', 'End_time', 'Duration'])
        ddfinal = mergeData(ddfinal)
        
#         print(ddfinal)
        ddfinal = ddfinal.drop(ddfinal[ddfinal['concept:name']=='waiting'].index)
    
        pm4py.write_xes(ddfinal, './%s/'%path + 'day%d.xes'%(i+1))
    #     Add daily routine to Final data
        daySimulation = daySimulation.append(ddfinal)
        
        # Create Event Log 3A

    #daySimulation = daySimulation.drop(daySimulation[daySimulation['concept:name']=='waiting'].index)
    #print(daySimulation)
    pm4py.write_xes(daySimulation, fileName)

In [147]:
def createEventLog3B(dayRange, fileName):
    path = './'+fileName.split('.')[0] 
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
    #xes_Index = ["case:concept:name","Activity" , "start_time", "End_time", "Duration"]
    daySimulation = pd.DataFrame()
    
#     Setting the start date (the first date in log)
    start_time = datetime(2022, 1, 1, 19, 0, 0, 0)

    for i in range(dayRange):
        # Create daily routine
        dayActivities = build3B()
#         dayActivities.append([wakeUp(), round(random.uniform(1, 15),2)])
        dd = pd.DataFrame(dayActivities)
        dd = pd.concat([dd[0].apply(lambda x: pd.Series(x)), dd[1]], axis=1)
        dd.columns = ['Activity', 'Min', 'Max', 'Duration']
        dd = dd.reindex(columns = ["Activity","Min", "Max", "start_time", "End_time", "Duration"]) 
        dd = dd.fillna("")
        #print(dd)
        # Randomly get time and calculate routine
#         start_time = datetime(2022, 1, i+1, 22, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S:%f')
#         Adding one day to the starting date
        start_time = start_time + timedelta(days=1)
    
        ddfinal = calculateDuration(dd, start_time)
        ddfinal = ddfinal.drop(columns =['Min', 'Max'])
        ddfinal = ddfinal.rename(columns={'Activity':'concept:name'})
        ddfinal = ddfinal.rename(columns={'End_time':'time:timestamp'})
        ddfinal.insert(loc=0, column='case:concept:name', value=i+1)
        ddfinal = mergeData(ddfinal)
        
        ddfinal = ddfinal.drop(ddfinal[ddfinal['concept:name']=='waiting'].index)
    
        pm4py.write_xes(ddfinal, './%s/'%path + 'day%d.xes'%(i+1))

    #     Add daily routine to Final data
        daySimulation = daySimulation.append(ddfinal)
        
        # Create Event Log 3A
    daySimulation = daySimulation.drop(daySimulation[daySimulation['concept:name']=='waiting'].index)
    pm4py.write_xes(daySimulation, fileName)

In [148]:
createEventLog3A(56, 'Evening3A.xes')

createEventLog3A(7, 'Evening3Ap.xes')
createEventLog3A(7, 'Evening3Ap2.xes')

createEventLog3B(7, 'Evening3B.xes')
createEventLog3B(7, 'Evening3B2.xes')

exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_54153/4270458896.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_54153/4270458896.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

...

exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_54153/2328106257.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/7c/r3slh6d1643d1hl26cgzfq5c0000gn/T/ipykernel_54153/2328106257.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daySimulation = daySimulation.append(ddfinal)


exporting log, completed traces ::   0%|          | 0/7 [00:00<?, ?it/s]

#### Anton & Yinggang
This repository is a part of our master's thesis project at DTU in the spring of 2022.\
The following items are available:

* Script
  * some
* Notebooks
* Applications
* Use cases
* Thesis

For further information, please contact us:\
Anton Freyr A

# ------------------------------